In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.interpolate import interpn
from scipy.stats import spearmanr
from sklearn.metrics.cluster import normalized_mutual_info_score
import os

In [ ]:
flat_dir = '/home/m/monroy/rubin-user/spec_flats/wl_dependence/outputs/'

In [ ]:
#flat_label = 'normalised_flat_{0}_{1}-{2}.fits'
flat_label = 'regular_flat_{0}_{1}-{2}.fits.gz'

In [ ]:
disperser = 'empty'

In [ ]:
filter_blue = 'SDSSg_65mm'
filter_red = 'SDSSz_65mm'
filter_0 = 'empty'
filter_blue_short = 'SDSSg'
filter_red_short = 'SDSSz'
filter_0_short = 'empty'

In [ ]:
id_blue = 2024041700090 #SDSSg
id_red = 2024041700119 #SDSSz
#id_red = 2024041700217 #SDSSr
id_0 = 2024041700177 #empty

In [ ]:
outdir = flat_dir

In [ ]:
flat_blue = fits.open(os.path.join(flat_dir,flat_label.format(id_blue,filter_blue,disperser)))
flat_red = fits.open(os.path.join(flat_dir,flat_label.format(id_red,filter_red,disperser)))
flat_0 = fits.open(os.path.join(flat_dir,flat_label.format(id_0,filter_0,disperser)))

In [ ]:
flat_shape = flat_red[1].data.shape
flat_shape

In [ ]:
ampli_coords = np.load(os.path.join(flat_dir,'LATISS_amplifier_coordinates.npy'),allow_pickle=True).ravel()[0]

In [ ]:
ampli_coords

In [ ]:
ampli_seq = ['C10','C11','C12','C13','C14','C15','C16','C17','C00','C01','C02','C03','C04','C05','C06','C07']

In [ ]:
#%matplotlib widget
fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(221)
im = ax1.imshow(flat_blue[1].data,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
ax1.set_title(filter_blue)
fig.colorbar(im,ax=ax1)

ax2 = fig.add_subplot(222)
im = ax2.imshow(flat_red[1].data,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
ax2.set_title(filter_red)
fig.colorbar(im,ax=ax2)

ax3 = fig.add_subplot(223)
im = ax3.imshow(flat_0[1].data,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
ax3.set_title(filter_0)
fig.colorbar(im,ax=ax3)

In [ ]:
fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(111)
im = ax1.imshow(flat_0[1].data,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
ax1.set_title(filter_0,fontsize=14)
fig.colorbar(im,ax=ax1)

for ampli_ in ampli_seq:
    coords_ = ampli_coords[ampli_]
    x_ = ((coords_[1]+coords_[0])/2.)-50.
    y_ = (coords_[3]+coords_[2])/2.
    ax1.text(x_,y_,ampli_,color='k')
    

plt.tight_layout()
plt.savefig(os.path.join(outdir,'auxtel_flat_{0}_{1}-{2}.png'.format(id_0,filter_0,disperser)))
#plt.close()

In [ ]:
#%matplotlib widget
for f_ in ['blue','red']:
    fig = plt.figure(figsize=(10,7))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(eval('flat_{0}'.format(f_))[1].data,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
    ax1.set_title(eval('filter_{0}'.format(f_)),fontsize=14)
    fig.colorbar(im,ax=ax1)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir,'auxtel_flat_{0}_{1}-{2}.png'.format(eval('id_{0}'.format(f_)),eval('filter_{0}'.format(f_)),disperser)))
    plt.close()


In [ ]:
flat_ratio = flat_blue[1].data/flat_red[1].data
flat_diff = flat_blue[1].data-flat_red[1].data

In [ ]:
fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(111)
im = ax1.imshow(flat_ratio,cmap="gray",origin='lower',vmin=0.96,vmax=1.04)
ax1.set_title(filter_blue+' / '+filter_red)
fig.colorbar(im,ax=ax1)
plt.tight_layout()


In [ ]:
percent_ = 2./100.
fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(111)
im = ax1.imshow(flat_diff,cmap="gray",origin='lower',vmin=-percent_,vmax=percent_)
ax1.set_title(filter_blue+' - '+filter_red)
fig.colorbar(im,ax=ax1)
plt.tight_layout()
plt.savefig(os.path.join(outdir,'auxtel_flat_diff_{0}-{1}_{2}_{3}.png'.format(filter_blue,filter_red,disperser,percent_)))

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,20))

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    if i<=7:
        row_ = 0
    else:
        row_ = 1
        i = i-8
    axs[row_,i].imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=0.98,vmax=1.02)
    axs[row_,i].set_title(ampli_)
    ampli_shape = flat_blue[1].data[y0_:y1_,x0_:x1_].shape

In [ ]:
ampli_shape

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,20))

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    if i<=7:
        row_ = 0
    else:
        row_ = 1
        i = i-8
    axs[row_,i].imshow(flat_diff[y0_:y1_,x0_:x1_],cmap="gray",origin='lower')
    axs[row_,i].set_title(ampli_)
    #ampli_shape = flat_blue[1].data[y0_:y1_,x0_:x1_].shape

### Define functions 

In [ ]:
def pearson_coeff(x,y):
    std_x = np.std(x)
    std_y = np.std(y)
    cov_xy = np.cov(x,y)[0,1]
    coeff = cov_xy/(std_x*std_y)
    return coeff

In [ ]:
#Limits for plotting histograms 
fval_min = 0.95
fval_max = 1.05

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(4, 4, figsize=(20,20))

i_ = 0
j = 0
for i0,ampli_ in enumerate(ampli_seq):
    print(ampli_)

    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dratio = dblue/dred

    rstd = np.std(dred)
    bstd = np.std(dblue)
    ratiostd = np.std(dratio)
    
    ########################################
    
    if i0>0 and (16-i0)%4==0:
        j = j+1
        i_ = 4*j
    i = i0-i_
    
    axs[j,i].hist(dred,bins=100,range=(fval_min,fval_max),histtype='step',color='r')
    axs[j,i].hist(dblue,bins=100,range=(fval_min,fval_max),histtype='step',color='b')
    nratio_, bratio_,_ = axs[j,i].hist(dratio,bins=100,range=(fval_min,fval_max),histtype='step',color='orange')
    axs[j,i].set_ylim(0.,np.max(nratio_)*1.6)
    axs[j,i].grid()
    axs[j,i].set_title(ampli_)
    axs[j,i].plot([],[],ls='-',label=r'$\sigma_{'+'{0}'.format(filter_blue_short)+'} = $'+'{0:.4f}'.format(rstd),color='b')
    axs[j,i].plot([],[],ls='-',label=r'$\sigma_{'+'{0}'.format(filter_red_short)+'} = $'+'{0:.4f}'.format(bstd),color='r')
    axs[j,i].plot([],[],ls='-',label=r'$\sigma_{ratio} = $'+'{0:.4f}'.format(ratiostd),color='orange')
    axs[j,i].legend(loc='upper left',fontsize=12)
    

In [ ]:
test_ampli = 'C14'

In [ ]:
x0_ = ampli_coords[test_ampli][0]
x1_ = ampli_coords[test_ampli][1]
y0_ = ampli_coords[test_ampli][2]
y1_ = ampli_coords[test_ampli][3]

dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
d0 = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()

In [ ]:
sindex0 = np.argsort(d0)

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(np.arange(len(d0)),d0[sindex0],ls='--',color='k')
ax.plot(np.arange(len(d0)),dblue[sindex0],color='b',alpha=0.4)
ax.plot(np.arange(len(d0)),dred[sindex0],color='r',alpha=0.4)

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.scatter(dblue,dred,marker='.',color='k',alpha=0.4)
ax.plot(d0,d0,ls='--',color='r')
ax.grid()
print(pearson_coeff(dblue,dred))
print(spearmanr(dblue,dred,axis=0)[0])
print(normalized_mutual_info_score(dblue,dred))

### Control plot

In [ ]:
_ = plt.hist2d(dblue,dred,bins=(500,500),range=[(fval_min,fval_max),(fval_min,fval_max)])
#plt.plot(x_,y_,color='r') 

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(4, 4, figsize=(22,19))

j = 0
for i,ampli_ in enumerate(ampli_seq):
    #print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dratio = dblue/dred
    
    dred_median = np.median(dred)
    dblue_median = np.median(dblue)
    
    dred_mean = np.mean(dred)
    dblue_mean = np.mean(dblue)
    
    dred_std = np.std(dred)
    dblue_std = np.std(dblue)

    coeff_rb = pearson_coeff(dred,dblue)
    
    if i%4==0 and i!=0:
        j += 1
    i = i-4*j
    #print(j,i)
    
    _,_,_,im = axs[j,i].hist2d(dblue,dred,bins=(500,500),range=[(fval_min,fval_max),(fval_min,fval_max)])
    #axs[j,i].set_xlim(xmin_blue,xmax_blue)
    #axs[j,i].set_ylim(xmin_red,xmax_red)
    axs[j,i].set_xlim(fval_min,fval_max)
    axs[j,i].set_ylim(fval_min,fval_max)
    axs[j,i].plot(np.linspace(fval_min,fval_max,100),np.linspace(fval_min,fval_max,100),color='r')        
    axs[j,i].plot([],[],ls='',label=r'$\rho = $'+'{0}'.format(coeff_rb.round(3)))
    axs[j,i].grid()
    if j==3:
        axs[j,i].set_xlabel(filter_red)#,fontsize=20)
    if i%4==0:
        axs[j,i].set_ylabel(filter_blue)#,fontsize=20)
    axs[j,i].set_title(ampli_)
    axs[j,i].legend(loc='lower right',fontsize=10)
    fig.colorbar(im,ax=axs[j,i])
    plt.savefig(os.path.join(outdir,'dispersion_plots.png'))
    

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(7,5))

i_ = 0
j = 0
ampli_ = 'C10'
i0 = 0
print(ampli_)

x0_ = ampli_coords[ampli_][0]
x1_ = ampli_coords[ampli_][1]
y0_ = ampli_coords[ampli_][2]
y1_ = ampli_coords[ampli_][3]

dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()


dsum = (dblue.flatten()+dred.flatten())/2.
_ = axs.hist(dsum,bins=300,range=(0.95,1.05))


In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(7,5))

i_ = 0
j = 0
ampli_ = 'C10'
i0 = 0
print(ampli_)

x0_ = ampli_coords[ampli_][0]
x1_ = ampli_coords[ampli_][1]
y0_ = ampli_coords[ampli_][2]
y1_ = ampli_coords[ampli_][3]

dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()

ddiff = (dblue.flatten()-dred.flatten())/2.
_ = axs.hist(ddiff,bins=300,range=(-0.04,0.04))

In [ ]:
ampli_ = 'C10'
x0_ = ampli_coords[ampli_][0]
x1_ = ampli_coords[ampli_][1]
y0_ = ampli_coords[ampli_][2]
y1_ = ampli_coords[ampli_][3]

dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()

# Create the main figure and axes
fig = plt.figure(figsize=(5, 5))
gs = fig.add_gridspec(3, 3)
ax_main = fig.add_subplot(gs[1:, :-1])

ax_right = fig.add_subplot(gs[1:, -1], sharey=ax_main)
ax_top = fig.add_subplot(gs[0, :-1], sharex=ax_main)

# Create the 2D histogram
nbins = 200
#hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(fval_min,fval_max),(fval_min,fval_max)])
 
# H needs to be rotated and flipped
H = np.rot90(H)
H = np.flipud(H)
 
# Mask zeros
Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero

im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
x_ = np.linspace(fval_min,fval_max,100)
y_ = x_
ax_main.plot(x_,y_,color='r') 
ax_main.grid()

coeff_rb_p = pearson_coeff(dred,dblue)
coeff_rb_s = spearmanr(dred,dblue,axis=0)[0]
ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))

axins1 = inset_axes(
    ax_main,
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="upper left",
)
axins1.xaxis.set_ticks_position("bottom")
fig.colorbar(im, cax=axins1, orientation="horizontal")

ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)

# Create the marginal histograms
ax_top.hist(dred, bins=500, range=(fval_min,fval_max), color='r')
ax_right.hist(dblue, bins=500, range=(fval_min,fval_max), orientation='horizontal', color='b')

std_red = np.std(dred)
std_blue = np.std(dblue)

ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))

ax_top.grid()
ax_right.grid()

# Remove ticks from marginal histograms
ax_top.tick_params(axis="x", labelbottom=False)
ax_right.tick_params(axis="y", labelleft=False)

ax_top.legend(loc="best",handlelength=0, handletextpad=0)
ax_right.legend(loc="best",handlelength=0, handletextpad=0)

# Set labels and title
ax_main.set_xlabel(filter_red_short)
ax_main.set_ylabel(filter_blue_short)

ax_top.set_title(ampli_)

# Print correlation coefficient
print("Pearson's correlation coefficient: {:.4f}".format(np.corrcoef(dred, dblue)[0, 1]))
print("Pearson's correlation coefficient: {:.4f}".format(spearmanr(dred,dblue,axis=0)[0]))

In [ ]:
x0_ = ampli_coords[ampli_][0]
x1_ = ampli_coords[ampli_][1]
y0_ = ampli_coords[ampli_][2]
y1_ = ampli_coords[ampli_][3]

dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()

# Create the main figure and axes
fig = plt.figure(figsize=(10, 8))
gs = fig.add_gridspec(3, 8)
ax_main = fig.add_subplot(gs[1:, :3])

ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)

# Create the 2D histogram
nbins = 200
#hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(fval_min,fval_max),(fval_min,fval_max)])
 
# H needs to be rotated and flipped
H = np.rot90(H)
H = np.flipud(H)
 
# Mask zeros
Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero

im, ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
x_ = np.linspace(fval_min,fval_max,100)
y_ = x_
ax_main.plot(x_,y_,color='r') 
ax_main.grid()

coeff_rb_p = pearson_coeff(dred,dblue)
coeff_rb_s = spearmanr(dred,dblue)[0]
ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))

axins1 = inset_axes(
    ax_main,
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="upper left",
)
axins1.xaxis.set_ticks_position("bottom")
fig.colorbar(im, cax=axins1, orientation="horizontal")

ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)

# Create the marginal histograms
ax_top.hist(dred, bins=500, range=(fval_min,fval_max), color='r')
ax_right.hist(dblue, bins=500, range=(fval_min,fval_max), orientation='horizontal', color='b')

ax_top.grid()
ax_right.grid()

std_red = np.std(dred)
std_blue = np.std(dblue)

ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))

# Remove ticks from marginal histograms
ax_top.tick_params(axis="x", labelbottom=False)
ax_right.tick_params(axis="y", labelleft=False)

ax_top.legend(loc="best",handlelength=0, handletextpad=0)
ax_right.legend(loc="best",handlelength=0, handletextpad=0)

# Set labels and title
ax_main.set_xlabel(filter_red_short)
ax_main.set_ylabel(filter_blue_short)

ax_top.set_title(ampli_)

ax_diff = fig.add_subplot(gs[0,4:])
ax_diff.hist(dblue-dred,bins=300,range=(-0.04,0.04))
ax_diff.grid()
ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))

ax_blue = fig.add_subplot(gs[1:, 5])
ax_red = fig.add_subplot(gs[1:, 6],sharey=ax_blue)
ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)

ax_red.tick_params(axis="y", labelleft=False)

ax_blue.set_title(filter_blue_short)
ax_red.set_title(filter_red_short)

'''
ax_cb = fig.add_subplot(gs[:, 7])
ax_cb.plot([],[])
fig.colorbar(im_red,cax=ax_cb,cmap='gray',fraction=0.046, pad=0.04)
'''
cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
fig.colorbar(im_red, cax=cax)

# Print correlation coefficient
print("Correlation coefficient: {:.4f}".format(np.corrcoef(dred, dblue)[0, 1]))

## Define function to do the plots 

### Plot general histograms and 2D distributions 

In [ ]:
def plot_histograms(ampli_,val_min,val_max,n_sigma=[1,2],saveplot=False):
    print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    
    # Create the main figure and axes
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(3, 8)
    ax_main = fig.add_subplot(gs[1:, :3])
    
    ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
    ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    y_ = x_
    ax_main.plot(x_,y_,color='r') 
    ax_main.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue,axis=0)[0]
    nmi_score = normalized_mutual_info_score(dred,dblue)
    ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    ax_main.plot([],[],ls='',label=r'$NMI = $'+'{0}'.format(nmi_score.round(3)))
    
    axins1 = inset_axes(
        ax_main,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    ax_main.set_xticklabels(ax_main.get_xticklabels(),rotation=45.,ha='right')
    ax_main.set_yticklabels(ax_main.get_yticklabels(),rotation=45.,ha='right')
    ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    # Create the marginal histograms
    ax_top.hist(dred, bins=500, range=(val_min,val_max), color='r')
    ax_right.hist(dblue, bins=500, range=(val_min,val_max), orientation='horizontal', color='b')

    mean_red = np.average(dred)
    mean_blue = np.average(dblue)
    std_red = np.std(dred)
    std_blue = np.std(dblue)
    
    color_sigma = ['k','grey','purple']
    
    for i,n_ in enumerate(n_sigma):
        ax_main.axhline(y=mean_blue+n_*std_blue,ls=':',color=color_sigma[i])
        ax_main.axhline(y=mean_blue-n_*std_blue,ls=':',color=color_sigma[i])
        ax_main.axvline(x=mean_red+n_*std_red,ls=':',color=color_sigma[i])
        ax_main.axvline(x=mean_red-n_*std_red,ls=':',color=color_sigma[i])
        
        ax_top.axvline(x=mean_red+n_*std_red,ls=':',color=color_sigma[i])
        ax_top.axvline(x=mean_red-n_*std_red,ls=':',color=color_sigma[i])
        ax_right.axhline(y=mean_blue+n_*std_blue,ls=':',color=color_sigma[i])
        ax_right.axhline(y=mean_blue-n_*std_blue,ls=':',color=color_sigma[i])
    
    
    ax_top.grid()
    ax_right.grid()
    
    ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
    ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))
    
    # Remove ticks from marginal histograms
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_right.tick_params(axis="y", labelleft=False)

    ax_top.legend(loc="best",handlelength=0, handletextpad=0)
    ax_right.legend(loc="best",handlelength=0, handletextpad=0)
    
    # Set labels and title
    ax_main.set_xlabel(filter_red_short)
    ax_main.set_ylabel(filter_blue_short)
    
    ax_top.set_title(ampli_)
    
    ax_diff = fig.add_subplot(gs[0,4:])
    ddiff = dblue-dred
    mean_diff = np.average(ddiff)
    std_diff = np.std(ddiff)

    ax_diff.hist(ddiff,bins=300,range=(-0.04,0.04))
    for i,n_ in enumerate(range(1,4)):
        ax_diff.axvline(x=mean_diff+n_*std_diff,ls=':',color=color_sigma[i],label='{0}'.format(n_)+r'$\sigma$')
        ax_diff.axvline(x=mean_diff-n_*std_diff,ls=':',color=color_sigma[i])
    ax_diff.grid()
    ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    ax_diff.legend(loc="best")

    ax_blue = fig.add_subplot(gs[1:, 5])
    ax_red = fig.add_subplot(gs[1:, 6],sharey=ax_blue)
    ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    
    ax_red.tick_params(axis="y", labelleft=False)
    
    ax_blue.set_title(filter_blue_short)
    ax_red.set_title(filter_red_short)
    
    cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
    fig.colorbar(im_red, cax=cax)
    if saveplot:
        plt.savefig(os.path.join(outdir,'dispersion_plot_detailed_0_ampli_{0}.png'.format(ampli_)))
    

In [ ]:
plot_histograms('C10',fval_min,fval_max,saveplot=True)

In [ ]:
for ampli_ in ampli_seq:
    plot_histograms(ampli_,fval_min,fval_max,n_sigma=[1])

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,18))

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred = flat_red[1].data[y0_:y1_,x0_:x1_]

    ddiff = dblue-dred
    
    if i<=7:
        row_ = 0
        col_ = i
    else:
        row_ = 1
        col_ = i-8
    
    im_ = axs[row_,col_].imshow(ddiff,cmap="gray",origin='lower',vmin=-0.04,vmax=0.04)
    xticks_ = axs[row_,col_].get_xticks()
    xticks_ = xticks_[1:-1].astype('int')
    axs[row_,col_].set_xticks(xticks_)
    axs[row_,col_].set_xticklabels(labels=xticks_+x0_)
    if col_>0:
        axs[row_,col_].tick_params(axis='y',labelleft=False)
    if col_==7:
        cax_ = fig.add_axes([axs[row_,col_].get_position().x1+0.01,axs[row_,col_].get_position().y0,0.02,axs[row_,col_].get_position().y1-axs[row_,col_].get_position().y0])
        fig.colorbar(im_,cax=cax_)
    axs[row_,col_].set_title(ampli_)
plt.savefig(os.path.join(outdir,'diff_image.png'))


### Plot histograms and 2D distributions in terms of $\sigma_{diff}$

In [ ]:
def plot_histograms_outliers(ampli_,val_min,val_max,n_sigma,interactive=False,saveplot=False):
    print(ampli_)

    if interactive:
        %matplotlib widget
    else:
        %matplotlib inline
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    
    # Create the main figure and axes
    fig = plt.figure(figsize=(12, 9))
    gs = fig.add_gridspec(3, 9)
    ax_main = fig.add_subplot(gs[1:, :3])
    
    ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
    ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    ax_main.plot(x_,x_,color='r') 
    ax_main.set_xlim(val_min,val_max)
    ax_main.set_ylim(val_min,val_max)
    ax_main.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue)[0]
    ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    
    axins1 = inset_axes(
        ax_main,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    
    ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    # Create the marginal histograms
    ax_top.hist(dred, bins=500, range=(val_min,val_max), color='r')
    ax_right.hist(dblue, bins=500, range=(val_min,val_max), orientation='horizontal', color='b')

    mean_red = np.average(dred)
    mean_blue = np.average(dblue)
    std_red = np.std(dred)
    std_blue = np.std(dblue)
    
    ax_top.grid()
    ax_right.grid()
    
    ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
    ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))
    
    # Remove ticks from marginal histograms
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_right.tick_params(axis="y", labelleft=False)

    ax_top.legend(loc="best",handlelength=0, handletextpad=0)
    ax_right.legend(loc="best",handlelength=0, handletextpad=0)
    
    # Set labels and title
    ax_main.set_xlabel(filter_red_short)
    ax_main.set_ylabel(filter_blue_short)
    
    ax_top.set_title(ampli_)
    
    ax_diff = fig.add_subplot(gs[0,4:])
    ddiff = flat_blue[1].data[y0_:y1_,x0_:x1_]-flat_red[1].data[y0_:y1_,x0_:x1_]
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    ax_diff.hist(ddiff.flatten(),bins=300,range=(-0.04,0.04))
    ax_diff.axvline(x=0.,ls='-',color='r')
    ax_diff.axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    ax_diff.axvline(x=mean_diff+n_sigma*std_diff,ls=':',color='k',label='{0}'.format(n_sigma)+r'$\sigma_{diff}$')
    ax_diff.axvline(x=mean_diff-n_sigma*std_diff,ls=':',color='k')
    ax_diff.grid()
    ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    ax_diff.legend(loc="best")

    #############################
    ax_main.plot(x_,x_+mean_diff,ls='--',color='k')
    ax_main.plot(x_,x_+mean_diff+n_sigma*std_diff,ls=':',color='k')
    ax_main.plot(x_,x_+mean_diff-n_sigma*std_diff,ls=':',color='k')

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-n_sigma*std_diff])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+n_sigma*std_diff])
    
    diff_mask = (ddiff>=xmin_diff)*(ddiff<=xmax_diff)
    #print(np.unique(diff_mask))
    
    
    color_dict_ = {0:'k',1:'white'}

    cm_ = ListedColormap(colors=list(color_dict_.values()))
    mask_labels_ = ['Out','In']
    
    ax_mask = fig.add_subplot(gs[1:, 5])
    im_mask = ax_mask.imshow(diff_mask,cmap=cm_,origin='lower')
    cax_ = fig.add_axes([ax_mask.get_position().x1+0.01,ax_mask.get_position().y0,0.02,ax_mask.get_position().y1-ax_mask.get_position().y0])
    cbar_ = fig.colorbar(im_mask, cax=cax_)

    yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
    yticks_ += (yticks_[1] - yticks_[0]) / 2
    cbar_.set_ticks(yticks_,labels=mask_labels_)
    ax_mask.set_title('{0}'.format(n_sigma)+r'$\sigma$')
    
    
    ax_blue = fig.add_subplot(gs[1:, 7],sharey=ax_mask)
    ax_red = fig.add_subplot(gs[1:, 8],sharey=ax_mask)
    ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)

    ax_blue.tick_params(axis="y", labelleft=False)
    ax_red.tick_params(axis="y", labelleft=False)
    
    ax_blue.set_title(filter_blue_short)
    ax_red.set_title(filter_red_short)
    
    cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
    fig.colorbar(im_red, cax=cax)
    if saveplot:
        plt.savefig(os.path.join(outdir,'dispersion_plot_detailed_1_{0}sigma_ampli_{1}.png'.format(n_sigma,ampli_)))
    

In [ ]:
plot_histograms_outliers('C10',fval_min,fval_max,n_sigma=1.5,interactive=False)

In [ ]:
for ampli_ in ampli_seq:
    plot_histograms_outliers(ampli_,fval_min,fval_max,n_sigma=1.5,saveplot=True)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,18))

color_dict_ = {0:'k',1:'white'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = ['Out','In']

n_sigma = 1.5
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred = flat_red[1].data[y0_:y1_,x0_:x1_]

    ddiff = dblue-dred
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-n_sigma*std_diff])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+n_sigma*std_diff])
    
    diff_mask = (ddiff>=xmin_diff)*(ddiff<=xmax_diff)
    
    if i<=7:
        row_ = 0
        col_ = i
    else:
        row_ = 1
        col_ = i-8
    
    im_ = axs[row_,col_].imshow(diff_mask,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
    xticks_ = axs[row_,col_].get_xticks()
    xticks_ = xticks_[1:-1].astype('int')
    axs[row_,col_].set_xticks(xticks_)
    axs[row_,col_].set_xticklabels(labels=xticks_+x0_)
    if col_>0:
        axs[row_,col_].tick_params(axis='y',labelleft=False)
    if col_==7:
        cax_ = fig.add_axes([axs[row_,col_].get_position().x1+0.01,axs[row_,col_].get_position().y0,0.02,axs[row_,col_].get_position().y1-axs[row_,col_].get_position().y0])
        cbar_ = fig.colorbar(im_,cax=cax_)
        yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
        yticks_ += (yticks_[1] - yticks_[0]) / 2
        cbar_.set_ticks(yticks_,labels=mask_labels_)
    axs[row_,col_].set_title(ampli_)
plt.savefig(os.path.join(outdir,'outlier_mask_image_{0}sigma.png'.format(n_sigma)))


In [ ]:
def plot_histograms_regions(ampli_,val_min,val_max,n_sigma,interactive=False,saveplot=True):
    print(ampli_)

    if interactive:
        %matplotlib widget
    else:
        %matplotlib inline
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    
    # Create the main figure and axes
    fig = plt.figure(figsize=(12, 9))
    gs = fig.add_gridspec(3, 9)
    ax_main = fig.add_subplot(gs[1:, :3])
    
    ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
    ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    ax_main.plot(x_,x_,color='r') 
    ax_main.set_xlim(val_min,val_max)
    ax_main.set_ylim(val_min,val_max)
    ax_main.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue)[0]
    ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    
    axins1 = inset_axes(
        ax_main,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    
    ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    # Create the marginal histograms
    ax_top.hist(dred, bins=500, range=(val_min,val_max), color='r')
    ax_right.hist(dblue, bins=500, range=(val_min,val_max), orientation='horizontal', color='b')

    mean_red = np.average(dred)
    mean_blue = np.average(dblue)
    std_red = np.std(dred)
    std_blue = np.std(dblue)
    
    ax_top.grid()
    ax_right.grid()
    
    ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
    ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))
    
    # Remove ticks from marginal histograms
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_right.tick_params(axis="y", labelleft=False)

    ax_top.legend(loc="best",handlelength=0, handletextpad=0)
    ax_right.legend(loc="best",handlelength=0, handletextpad=0)
    
    # Set labels and title
    ax_main.set_xlabel(filter_red_short)
    ax_main.set_ylabel(filter_blue_short)
    
    ax_top.set_title(ampli_)
    
    ax_diff = fig.add_subplot(gs[0,4:])
    ddiff = flat_blue[1].data[y0_:y1_,x0_:x1_]-flat_red[1].data[y0_:y1_,x0_:x1_]
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    ax_diff.hist(ddiff.flatten(),bins=300,range=(-0.04,0.04))
    ax_diff.axvline(x=0.,ls='-',color='r')
    ax_diff.axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    ax_diff.axvline(x=mean_diff+n_sigma*std_diff,ls=':',color='k',label='{0}'.format(n_sigma)+r'$\sigma_{diff}$')
    ax_diff.axvline(x=mean_diff-n_sigma*std_diff,ls=':',color='k')
    ax_diff.grid()
    ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    ax_diff.legend(loc="best")

    #############################
    ax_main.plot(x_,x_+mean_diff,ls='--',color='k')
    ax_main.plot(x_,x_+mean_diff+n_sigma*std_diff,ls=':',color='k')
    ax_main.plot(x_,x_+mean_diff-n_sigma*std_diff,ls=':',color='k')

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-n_sigma*std_diff])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+n_sigma*std_diff])

    sides_mask = np.zeros(ampli_shape).astype('int')
    left_mask = (ddiff>=xmin_diff)*(ddiff<=mean_diff)
    right_mask = (ddiff>mean_diff)*(ddiff<=xmax_diff)
    
    sides_mask[left_mask] = -1
    sides_mask[right_mask] = 1
    
    color_dict_ = {-1:'r',0:'k',1:'b'}

    cm_ = ListedColormap(colors=list(color_dict_.values()))
    mask_labels_ = [r'$[\mu_{diff}-$'+'{0}'.format(n_sigma)+r'$\sigma_{diff}, \mu_{diff} ]$',
                    'Out',
                    r'$[\mu_{diff}, \mu_{diff}+$'+'{0}'.format(n_sigma)+r'$\sigma_{diff}]$']
    
    ax_mask = fig.add_subplot(gs[1:, 5])
    im_mask = ax_mask.imshow(sides_mask,cmap=cm_,origin='lower')
    cax_ = fig.add_axes([ax_mask.get_position().x1+0.01,ax_mask.get_position().y0,0.02,ax_mask.get_position().y1-ax_mask.get_position().y0])
    cbar_ = fig.colorbar(im_mask, cax=cax_)

    yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
    #yticks_ += (yticks_[1] - yticks_[0]) / 2
    yticks_ = [yticks_[1],yticks_[1]+(yticks_[1] - yticks_[0]) / 2,yticks_[2]+(yticks_[1] - yticks_[0])]
    cbar_.set_ticks(yticks_)
    cbar_.set_ticklabels(ticklabels=mask_labels_,rotation=90)
    
    ax_blue = fig.add_subplot(gs[1:, 7],sharey=ax_mask)
    ax_red = fig.add_subplot(gs[1:, 8],sharey=ax_mask)
    ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)

    ax_blue.tick_params(axis="y", labelleft=False)
    ax_red.tick_params(axis="y", labelleft=False)
    
    ax_blue.set_title(filter_blue_short)
    ax_red.set_title(filter_red_short)
    
    cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
    fig.colorbar(im_red, cax=cax)
    if saveplot:
        plt.savefig(os.path.join(outdir,'dispersion_plot_detailed_2_{0}sigma_ampli_{1}.png'.format(n_sigma,ampli_)))
    

In [ ]:
plot_histograms_regions('C10',fval_min,fval_max,n_sigma=1.5,interactive=False)

In [ ]:
for ampli_ in ampli_seq:
    plot_histograms_regions(ampli_,fval_min,fval_max,n_sigma=1.5,saveplot=True)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,18))

color_dict_ = {-1:'r',0:'k',1:'b'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = [r'$[\mu_{diff}-$'+'{0}'.format(n_sigma)+r'$\sigma_{diff}, \mu_{diff} ]$',
                'Out',
                r'$[\mu_{diff}, \mu_{diff}+$'+'{0}'.format(n_sigma)+r'$\sigma_{diff}]$']

n_sigma = 1.5
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred = flat_red[1].data[y0_:y1_,x0_:x1_]

    ddiff = dblue-dred
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-n_sigma*std_diff])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+n_sigma*std_diff])
    
    sides_mask = np.zeros(ampli_shape).astype('int')
    left_mask = (ddiff>=xmin_diff)*(ddiff<=mean_diff)
    right_mask = (ddiff>mean_diff)*(ddiff<=xmax_diff)
    
    sides_mask[left_mask] = -1
    sides_mask[right_mask] = 1
    
    if i<=7:
        row_ = 0
        col_ = i
    else:
        row_ = 1
        col_ = i-8
    
    im_ = axs[row_,col_].imshow(sides_mask,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
    xticks_ = axs[row_,col_].get_xticks()
    xticks_ = xticks_[1:-1].astype('int')
    axs[row_,col_].set_xticks(xticks_)
    axs[row_,col_].set_xticklabels(labels=xticks_+x0_)
    if col_>0:
        axs[row_,col_].tick_params(axis='y',labelleft=False)
    if col_==7:
        cax_ = fig.add_axes([axs[row_,col_].get_position().x1+0.01,axs[row_,col_].get_position().y0,0.02,axs[row_,col_].get_position().y1-axs[row_,col_].get_position().y0])
        cbar_ = fig.colorbar(im_,cax=cax_)
        yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
        yticks_ = [yticks_[1],yticks_[1]+(yticks_[1] - yticks_[0]) / 2,yticks_[2]+(yticks_[1] - yticks_[0])]
        cbar_.set_ticks(yticks_)
        cbar_.set_ticklabels(ticklabels=mask_labels_,rotation=90)
    axs[row_,col_].set_title(ampli_)
plt.savefig(os.path.join(outdir,'regions_image_{0}sigma.png'.format(n_sigma)))


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,20))

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]

    dred = flat_red[1].data[y0_:y1_,x0_:x1_]
    
    nsig = 3.

    dred_mean = np.mean(dred.flatten())
    
    dred_std = np.std(dred.flatten())
    
    xmin_red = np.max(dred[dred<=dred_mean-nsig*dred_std])
    xmax_red = np.min(dred[dred>=dred_mean+nsig*dred_std])
    
    mask_ = (dred>=xmin_red)*(dred<=xmax_red)
    
    if i<=7:
        row_ = 0
    else:
        row_ = 1
        i = i-8
    
    axs[row_,i].imshow(mask_,cmap="gray",origin='lower')#,vmin=0,vmax=1)
    axs[row_,i].set_title(ampli_)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,20))

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]

    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    
    nsig = 3.

    dblue_mean = np.mean(dblue.flatten())
    
    dblue_std = np.std(dblue.flatten())
    
    xmin_blue = np.max(dblue[dblue<=dblue_mean-nsig*dblue_std])
    xmax_blue = np.min(dblue[dblue>=dblue_mean+nsig*dblue_std])
    
    mask_ = (dblue>=xmin_blue)*(dblue<=xmax_blue)
    
    if i<=7:
        row_ = 0
    else:
        row_ = 1
        i = i-8
    
    axs[row_,i].imshow(mask_,cmap="gray",origin='lower')#,vmin=0,vmax=1)
    axs[row_,i].set_title(ampli_)

### Plot histograms and 2D distributions in terms of difference with respect to $\mu_0$

In [ ]:
def plot_histograms_diffper(ampli_,val_min,val_max,percent,interactive=False,saveplot=False):
    print(ampli_)

    if interactive:
        %matplotlib widget
    else:
        %matplotlib inline
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    d0 = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    # Create the main figure and axes
    fig = plt.figure(figsize=(12, 9))
    gs = fig.add_gridspec(3, 9)
    ax_main = fig.add_subplot(gs[1:, :3])
    
    ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
    ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    ax_main.plot(x_,x_,color='r') 
    ax_main.set_xlim(val_min,val_max)
    ax_main.set_ylim(val_min,val_max)
    ax_main.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue)[0]
    nmi_score = normalized_mutual_info_score(dred,dblue)
    ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    ax_main.plot([],[],ls='',label=r'$NMI = $'+'{0}'.format(nmi_score.round(3)))
    
    axins1 = inset_axes(
        ax_main,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    
    ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    # Create the marginal histograms
    ax_top.hist(dred, bins=500, range=(val_min,val_max), color='r')
    ax_right.hist(dblue, bins=500, range=(val_min,val_max), orientation='horizontal', color='b')

    mean_red = np.average(dred)
    mean_blue = np.average(dblue)
    std_red = np.std(dred)
    std_blue = np.std(dblue)

    mean_0 = np.average(d0)
    
    ax_top.grid()
    ax_right.grid()
    
    ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
    ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))
    
    # Remove ticks from marginal histograms
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_right.tick_params(axis="y", labelleft=False)

    ax_top.legend(loc="best",handlelength=0, handletextpad=0)
    ax_right.legend(loc="best",handlelength=0, handletextpad=0)
    
    # Set labels and title
    ax_main.set_xlabel(filter_red_short)
    ax_main.set_ylabel(filter_blue_short)
    
    ax_top.set_title(ampli_)
    
    ax_diff = fig.add_subplot(gs[0,4:])
    ddiff = flat_blue[1].data[y0_:y1_,x0_:x1_]-flat_red[1].data[y0_:y1_,x0_:x1_]
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    ax_diff.hist(ddiff.flatten(),bins=300,range=(-0.04,0.04))
    ax_diff.axvline(x=0.,ls='-',color='r')
    ax_diff.axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    ax_diff.axvline(x=mean_diff+mean_0*percent/100.,ls=':',color='k',label=r'$\mu_{diff} \pm $'+'{0}'.format(percent/100.)+r'$\mu_0$')
    ax_diff.axvline(x=mean_diff-mean_0*percent/100.,ls=':',color='k')
    ax_diff.grid()
    ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    ax_diff.legend(loc="best")

    #############################
    ax_main.plot(x_,x_+mean_diff,ls='--',color='k')
    ax_main.plot(x_,x_+mean_diff+mean_0*percent/100.,ls=':',color='k')
    ax_main.plot(x_,x_+mean_diff-mean_0*percent/100.,ls=':',color='k')

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-mean_0*percent/100.])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+mean_0*percent/100.])
    #print(xmin_diff,xmax_diff)
    
    diff_mask = (ddiff>=xmin_diff)*(ddiff<=xmax_diff)
    #print(np.unique(diff_mask))
    
    
    color_dict_ = {0:'k',1:'white'}

    cm_ = ListedColormap(colors=list(color_dict_.values()))
    mask_labels_ = ['Out','In']
    
    ax_mask = fig.add_subplot(gs[1:, 5])
    im_mask = ax_mask.imshow(diff_mask,cmap=cm_,origin='lower')
    cax_ = fig.add_axes([ax_mask.get_position().x1+0.01,ax_mask.get_position().y0,0.02,ax_mask.get_position().y1-ax_mask.get_position().y0])
    cbar_ = fig.colorbar(im_mask, cax=cax_)

    yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
    yticks_ += (yticks_[1] - yticks_[0]) / 2
    cbar_.set_ticks(yticks_,labels=mask_labels_)
    ax_mask.set_title('{0}'.format(percent/100.)+r'$\mu_0$')
    
    
    ax_blue = fig.add_subplot(gs[1:, 7],sharey=ax_mask)
    ax_red = fig.add_subplot(gs[1:, 8],sharey=ax_mask)
    ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)

    ax_blue.tick_params(axis="y", labelleft=False)
    ax_red.tick_params(axis="y", labelleft=False)
    
    ax_blue.set_title(filter_blue_short)
    ax_red.set_title(filter_red_short)
    
    cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
    fig.colorbar(im_red, cax=cax)
    if saveplot:
        plt.savefig(os.path.join(outdir,'dispersion_plot_detailed_4_{0}percent_ampli_{1}.png'.format(percent,ampli_)))
    

In [ ]:
fval_min = 0.9
fval_max = 1.1

In [ ]:
plot_histograms_diffper('C00',fval_min,fval_max,percent=1.,interactive=False)

In [ ]:
for ampli_ in ampli_seq:
    plot_histograms_diffper(ampli_,fval_min,fval_max,percent=1.0,saveplot=True)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,18))

color_dict_ = {0:'k',1:'white'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = ['Out','In']

percent = 2.0
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred = flat_red[1].data[y0_:y1_,x0_:x1_]
    d0 = flat_0[1].data[y0_:y1_,x0_:x1_]

    ddiff = dblue-dred
    mean_diff = np.average(ddiff.flatten())
    std_diff = np.std(ddiff.flatten())

    mean_0 = np.average(d0)

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-mean_0*percent/100.])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+mean_0*percent/100.])
    
    diff_mask = (ddiff>=xmin_diff)*(ddiff<=xmax_diff)
    
    if i<=7:
        row_ = 0
        col_ = i
    else:
        row_ = 1
        col_ = i-8
    
    im_ = axs[row_,col_].imshow(diff_mask,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
    xticks_ = axs[row_,col_].get_xticks()
    xticks_ = xticks_[1:-1].astype('int')
    axs[row_,col_].set_xticks(xticks_)
    axs[row_,col_].set_xticklabels(labels=xticks_+x0_)
    if col_>0:
        axs[row_,col_].tick_params(axis='y',labelleft=False)
    if col_==7:
        cax_ = fig.add_axes([axs[row_,col_].get_position().x1+0.01,axs[row_,col_].get_position().y0,0.02,axs[row_,col_].get_position().y1-axs[row_,col_].get_position().y0])
        cbar_ = fig.colorbar(im_,cax=cax_)
        yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
        yticks_ += (yticks_[1] - yticks_[0]) / 2
        cbar_.set_ticks(yticks_,labels=mask_labels_)
    axs[row_,col_].set_title(ampli_)
plt.savefig(os.path.join(outdir,'outlier_mask_image_{0:.2f}percent.png'.format(percent)))


In [ ]:
def plot_histograms_diffper_regions(ampli_,val_min,val_max,percent,interactive=False,saveplot=True):
    print(ampli_)

    if interactive:
        %matplotlib widget
    else:
        %matplotlib inline
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    d0 = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    # Create the main figure and axes
    fig = plt.figure(figsize=(12, 9))
    gs = fig.add_gridspec(3, 9)
    ax_main = fig.add_subplot(gs[1:, :3])
    
    ax_right = fig.add_subplot(gs[1:, 3], sharey=ax_main)
    ax_top = fig.add_subplot(gs[0, :3], sharex=ax_main)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax_main.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    ax_main.plot(x_,x_,color='r') 
    ax_main.set_xlim(val_min,val_max)
    ax_main.set_ylim(val_min,val_max)
    ax_main.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue)[0]
    ax_main.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax_main.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    
    axins1 = inset_axes(
        ax_main,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    
    ax_main.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    # Create the marginal histograms
    ax_top.hist(dred, bins=500, range=(val_min,val_max), color='r')
    ax_right.hist(dblue, bins=500, range=(val_min,val_max), orientation='horizontal', color='b')

    mean_red = np.average(dred)
    mean_blue = np.average(dblue)
    std_red = np.std(dred)
    std_blue = np.std(dblue)

    mean_0 = np.average(d0)
    
    ax_top.grid()
    ax_right.grid()
    
    ax_top.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_red))
    ax_right.plot([],[],ls='',label=r'$\sigma = $'+'{0:.2f}'.format(std_blue))
    
    # Remove ticks from marginal histograms
    ax_top.tick_params(axis="x", labelbottom=False)
    ax_right.tick_params(axis="y", labelleft=False)

    ax_top.legend(loc="best",handlelength=0, handletextpad=0)
    ax_right.legend(loc="best",handlelength=0, handletextpad=0)
    
    # Set labels and title
    ax_main.set_xlabel(filter_red_short)
    ax_main.set_ylabel(filter_blue_short)
    
    ax_top.set_title(ampli_)
    
    ax_diff = fig.add_subplot(gs[0,4:])
    ddiff = flat_blue[1].data[y0_:y1_,x0_:x1_]-flat_red[1].data[y0_:y1_,x0_:x1_]
    mean_diff = np.average(ddiff.flatten())

    ax_diff.hist(ddiff.flatten(),bins=300,range=(-0.04,0.04))
    ax_diff.axvline(x=0.,ls='-',color='r')
    ax_diff.axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    ax_diff.axvline(x=mean_diff+mean_0*percent/100.,ls=':',color='k',label=r'$\mu_{diff} \pm $'+'{0}'.format(percent/100.)+r'$\mu_0$')
    ax_diff.axvline(x=mean_diff-mean_0*percent/100.,ls=':',color='k')
    ax_diff.grid()
    ax_diff.set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    ax_diff.legend(loc="best")

    #############################
    ax_main.plot(x_,x_+mean_diff,ls='--',color='k')
    ax_main.plot(x_,x_+mean_diff+mean_0*percent/100.,ls=':',color='k')
    ax_main.plot(x_,x_+mean_diff-mean_0*percent/100.,ls=':',color='k')

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-mean_0*percent/100.])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+mean_0*percent/100.])

    sides_mask = np.zeros(ampli_shape).astype('int')
    left_mask = (ddiff>=xmin_diff)*(ddiff<=mean_diff)
    right_mask = (ddiff>mean_diff)*(ddiff<=xmax_diff)
    
    sides_mask[left_mask] = -1
    sides_mask[right_mask] = 1
    
    color_dict_ = {-1:'r',0:'k',1:'b'}

    cm_ = ListedColormap(colors=list(color_dict_.values()))
    mask_labels_ = [r'$[\mu_{diff}-$'+'{0}'.format(percent/100.)+r'$\mu_0, \mu_{diff} ]$',
                    'Out',
                    r'$[\mu_{diff}, \mu_{diff}+$'+'{0}'.format(percent/100.)+r'$\mu_0]$']
    
    ax_mask = fig.add_subplot(gs[1:, 5])
    im_mask = ax_mask.imshow(sides_mask,cmap=cm_,origin='lower')
    cax_ = fig.add_axes([ax_mask.get_position().x1+0.01,ax_mask.get_position().y0,0.02,ax_mask.get_position().y1-ax_mask.get_position().y0])
    cbar_ = fig.colorbar(im_mask, cax=cax_)

    yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
    #yticks_ += (yticks_[1] - yticks_[0]) / 2
    yticks_ = [yticks_[1],yticks_[1]+(yticks_[1] - yticks_[0]) / 2,yticks_[2]+(yticks_[1] - yticks_[0])]
    cbar_.set_ticks(yticks_)
    cbar_.set_ticklabels(ticklabels=mask_labels_,rotation=90)
    
    ax_blue = fig.add_subplot(gs[1:, 7],sharey=ax_mask)
    ax_red = fig.add_subplot(gs[1:, 8],sharey=ax_mask)
    ax_blue.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
    im_red = ax_red.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)

    ax_blue.tick_params(axis="y", labelleft=False)
    ax_red.tick_params(axis="y", labelleft=False)
    
    ax_blue.set_title(filter_blue_short)
    ax_red.set_title(filter_red_short)
    
    cax = fig.add_axes([ax_red.get_position().x1+0.01,ax_red.get_position().y0,0.02,ax_red.get_position().y1-ax_red.get_position().y0])
    fig.colorbar(im_red, cax=cax)
    if saveplot:
        plt.savefig(os.path.join(outdir,'dispersion_plot_detailed_5_{0}sigma_ampli_{1}.png'.format(n_sigma,ampli_)))
    

In [ ]:
plot_histograms_diffper_regions('C10',fval_min,fval_max,percent=1.,interactive=False)

In [ ]:
for ampli_ in ampli_seq:
    plot_histograms_diffper_regions(ampli_,fval_min,fval_max,percent=2.0,saveplot=True)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,8,figsize=(20,18))

color_dict_ = {-1:'r',0:'k',1:'b'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = [r'$[\mu_{diff}-$'+'{0}'.format(percent/100.)+r'$\mu_0, \mu_{diff} ]$',
                    'Out',
                    r'$[\mu_{diff}, \mu_{diff}+$'+'{0}'.format(percent/100.)+r'$\mu_0]$']

percent = 2.0
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred = flat_red[1].data[y0_:y1_,x0_:x1_]
    d0 = flat_0[1].data[y0_:y1_,x0_:x1_]

    ddiff = dblue-dred
    mean_diff = np.average(ddiff.flatten())
    mean_0 = np.average(d0)

    #############################
    xmin_diff = np.max(ddiff[ddiff<=mean_diff-mean_0*percent/100.])
    xmax_diff = np.min(ddiff[ddiff>=mean_diff+mean_0*percent/100.])
    
    sides_mask = np.zeros(ampli_shape).astype('int')
    left_mask = (ddiff>=xmin_diff)*(ddiff<=mean_diff)
    right_mask = (ddiff>mean_diff)*(ddiff<=xmax_diff)
    
    sides_mask[left_mask] = -1
    sides_mask[right_mask] = 1
    
    if i<=7:
        row_ = 0
        col_ = i
    else:
        row_ = 1
        col_ = i-8
    
    im_ = axs[row_,col_].imshow(sides_mask,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
    xticks_ = axs[row_,col_].get_xticks()
    xticks_ = xticks_[1:-1].astype('int')
    axs[row_,col_].set_xticks(xticks_)
    axs[row_,col_].set_xticklabels(labels=xticks_+x0_)
    if col_>0:
        axs[row_,col_].tick_params(axis='y',labelleft=False)
    if col_==7:
        cax_ = fig.add_axes([axs[row_,col_].get_position().x1+0.01,axs[row_,col_].get_position().y0,0.02,axs[row_,col_].get_position().y1-axs[row_,col_].get_position().y0])
        cbar_ = fig.colorbar(im_,cax=cax_)
        yticks_ = np.linspace(*cbar_.ax.get_ylim(), cm_.N+1)[:-1]
        yticks_ = [yticks_[1],yticks_[1]+(yticks_[1] - yticks_[0]) / 2,yticks_[2]+(yticks_[1] - yticks_[0])]
        cbar_.set_ticks(yticks_)
        cbar_.set_ticklabels(ticklabels=mask_labels_,rotation=90)
    axs[row_,col_].set_title(ampli_)
plt.savefig(os.path.join(outdir,'regions_image_{0:.2f}percent.png'.format(percent)))


## For the paper 

### Correlation coefficients / scores 

In [ ]:
corr_dict = {}
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()

    rp_ = pearson_coeff(dblue,dred)
    rs_ = spearmanr(dblue,dred,axis=0)[0]
    nmi_ = normalized_mutual_info_score(dblue,dred)
    
    corr_dict[ampli_] = (rp_,rs_,nmi_)

rp_all = pearson_coeff(flat_blue[1].data.flatten(),flat_red[1].data.flatten())
rs_all = spearmanr(flat_blue[1].data.flatten(),flat_red[1].data.flatten(),axis=0)[0]
nmi_all = normalized_mutual_info_score(flat_blue[1].data.flatten(),flat_red[1].data.flatten())
corr_dict['all'] = (rp_all,rs_all,nmi_all)


In [ ]:
fig = plt.figure(figsize=(12,7))
ax = fig.add_subplot(111)

for i,ampli_ in enumerate(corr_dict.keys()):
    ax.scatter(i,corr_dict[ampli_][0],marker='o',color='b',zorder=2)
    ax.scatter(i,corr_dict[ampli_][1],marker='s',color='r',zorder=2)
    ax.scatter(i,corr_dict[ampli_][2],marker='^',color='orange',zorder=2)
ax.plot([],[],ls='',marker='o',color='b',label=r'$\rho_p$')
ax.plot([],[],ls='',marker='s',color='r',label=r'$\rho_s$')
ax.plot([],[],ls='',marker='^',color='orange',label=r'$NMI$')
ax.grid()
ax.set_xlabel('Amplifier',fontsize=14)
ax.set_ylabel('Correlation coefficient / score',fontsize=14)
ax.set_xticks(np.arange(len(corr_dict.keys())))
ax.set_xticklabels(corr_dict.keys(),fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=12)
ax.legend(loc="best",fontsize=14)
#plt.tight_layout()
plt.savefig(os.path.join(outdir,'correlation_coeffs_{0}_{1}_{2}.png'.format(filter_blue,filter_red,disperser)))

### Dispersion plots 

In [ ]:
def make_2d_histogram(dred,dblue,d0,percent,val_min,val_max,fig,ax,title,do_cax=False):
    
    std_red = np.std(dred)
    std_blue = np.std(dblue)
    
    mean_0 = np.average(d0)
    
    ddiff = dblue-dred
    mean_diff = np.average(ddiff)
    
    # Create the 2D histogram
    nbins = 200
    #hist, x_edges, y_edges, im = ax_main.hist2d(dred, dblue, bins=nbins, range=[(fval_min,fval_max),(fval_min,fval_max)], cmap='jet')
    H, xedges, yedges = np.histogram2d(dred,dblue,bins=500,range=[(val_min,val_max),(val_min,val_max)])
     
    # H needs to be rotated and flipped
    H = np.rot90(H)
    H = np.flipud(H)
     
    # Mask zeros
    Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
    
    im = ax.pcolormesh(xedges,yedges,Hmasked,cmap='jet')
    x_ = np.linspace(val_min,val_max,100)
    ax.plot(x_,x_,color='r') 
    ax.set_xlim(val_min,val_max)
    ax.set_ylim(val_min,val_max)
    ax.grid()
    
    coeff_rb_p = pearson_coeff(dred,dblue)
    coeff_rb_s = spearmanr(dred,dblue)[0]
    nmi_score = normalized_mutual_info_score(dred,dblue)
    ax.plot([],[],ls='',label=r'$\rho_{p} = $'+'{0}'.format(coeff_rb_p.round(3)))
    ax.plot([],[],ls='',label=r'$\rho_{s} = $'+'{0}'.format(coeff_rb_s.round(3)))
    ax.plot([],[],ls='',label=r'$NMI = $'+'{0}'.format(nmi_score.round(3)))
    
    ax.plot(x_,x_+mean_diff,ls='--',color='k')
    ax.plot(x_,x_+mean_diff+mean_0*percent/100.,ls=':',color='k')
    ax.plot(x_,x_+mean_diff-mean_0*percent/100.,ls=':',color='k')
    
    ax.set_xlabel(filter_red_short)
    ax.set_ylabel(filter_blue_short)

    ax.set_xticks(ax.get_xticks(),ax.get_xticklabels())
    ax.set_yticks(ax.get_yticks(),ax.get_yticklabels())
    #ax.set_xticklabels(ax.get_xticklabels(),rotation=45.,ha='right')
    #ax.set_yticklabels(ax.get_yticklabels(),rotation=45.,ha='right')
    
    axins1 = inset_axes(
        ax,
        width="50%",  # width: 50% of parent_bbox width
        height="5%",  # height: 5%
        loc="upper left",
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(im, cax=axins1, orientation="horizontal")
    
    ax.legend(loc="lower right",handlelength=0, handletextpad=0)
    
    if do_cax:
        cax_red = fig.add_axes([ax.get_position().x0,ax.get_position().y1+0.02,ax.get_position().x1-ax.get_position().x0,0.15])
        _ = cax_red.hist(dred,bins=500, range=(fval_min,fval_max), color='r')
        cax_red.set_xlim(fval_min,fval_max)
        cax_red.grid()
        
        cax_blue = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.15,ax.get_position().y1-ax.get_position().y0])
        _ = cax_blue.hist(dblue,bins=500, range=(fval_min,fval_max), orientation='horizontal', color='b')
        cax_blue.set_ylim(fval_min,fval_max)
        cax_blue.grid()
        
        cax_red.plot([],[],ls='',label=r'$\sigma_{red} = $'+'{0:.2f}'.format(std_red))
        cax_blue.plot([],[],ls='',label=r'$\sigma_{blue} = $'+'{0:.2f}'.format(std_blue))
        
        # Remove ticks from marginal histograms
        cax_red.tick_params(axis="x", labelbottom=False)
        cax_blue.tick_params(axis="y", labelleft=False)
        
        cax_red.legend(loc="best",handlelength=0, handletextpad=0)
        cax_blue.legend(loc="best",handlelength=0, handletextpad=0)
        cax_red.set_title(title)
    else:
        ax.set_title(title)

    return ax
    

In [ ]:
%matplotlib inline
nrows = 4
ncols = 2
fig, axs = plt.subplots(nrows, ncols, figsize=(10,14))

percent = 1.0

j = 0
for i,ampli_ in enumerate(ampli_seq[:8]):
    #print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    if i%ncols==0 and i!=0:
        j += 1
    i = i-ncols*j
    #print(j,i)
    
    make_2d_histogram(dred_,dblue_,d0_,percent,fval_min,fval_max,fig,axs[j,i],ampli_)

    axs[j,i].set_xticklabels(axs[j,i].get_xticklabels(),rotation=45.,ha='right')
    axs[j,i].set_yticklabels(axs[j,i].get_yticklabels(),rotation=45.,ha='right')
    if j!=nrows-1:
        axs[j,i].set_xlabel('')
        axs[j,i].set_xticklabels([])
    if i!=0:
        axs[j,i].set_ylabel('')
        axs[j,i].set_yticklabels([])
        
plt.savefig(os.path.join(outdir,'dispersion_plots_up_paper.png'))


In [ ]:
%matplotlib inline
nrows = 4
ncols = 2
fig, axs = plt.subplots(nrows, ncols, figsize=(10,14))

percent = 1.0

j = 0
for i,ampli_ in enumerate(ampli_seq[8:]):
    #print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    if i%ncols==0 and i!=0:
        j += 1
    i = i-ncols*j
    #print(j,i)
    
    make_2d_histogram(dred_,dblue_,d0_,percent,fval_min,fval_max,fig,axs[j,i],ampli_)

    axs[j,i].set_xticklabels(axs[j,i].get_xticklabels(),rotation=45.,ha='right')
    axs[j,i].set_yticklabels(axs[j,i].get_yticklabels(),rotation=45.,ha='right')
    if j!=nrows-1:
        axs[j,i].set_xlabel('')
        axs[j,i].set_xticklabels([])
    if i!=0:
        axs[j,i].set_ylabel('')
        axs[j,i].set_yticklabels([])
        
plt.savefig(os.path.join(outdir,'dispersion_plots_bottom_paper.png'))


### Difference histograms 

In [ ]:
%matplotlib inline
nrows = 4
ncols = 2
fig, axs = plt.subplots(nrows, ncols, figsize=(10,14))

percent = 1.0

j = 0
for i,ampli_ in enumerate(ampli_seq[:8]):
    #print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    ddiff_ = dblue_-dred_
    mean_diff = np.average(ddiff_)
    mean_0 = np.average(d0_)

    if i%ncols==0 and i!=0:
        j += 1
    i = i-ncols*j
    #print(j,i)
    
    axs[j,i].hist(ddiff_,bins=300,range=(-0.04,0.04))
    axs[j,i].axvline(x=0.,ls='-',color='r')
    axs[j,i].axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    axs[j,i].axvline(x=mean_diff+mean_0*percent/100.,ls=':',color='k',label=r'$\mu_{diff} \pm $'+'{0}'.format(percent/100.)+r'$\mu_0$')
    axs[j,i].axvline(x=mean_diff-mean_0*percent/100.,ls=':',color='k')
    axs[j,i].grid()
    axs[j,i].set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    axs[j,i].set_ylabel('Number of pixels')
    axs[j,i].set_title(ampli_)
    axs[j,i].legend(loc="best")
    
    if j!=nrows-1:
        axs[j,i].set_xlabel('')
    if i!=0:
        axs[j,i].set_ylabel('')
        
plt.savefig(os.path.join(outdir,'diff_histograms_up_paper.png'))


In [ ]:
%matplotlib inline
nrows = 4
ncols = 2
fig, axs = plt.subplots(nrows, ncols, figsize=(10,14))

percent = 1.0

j = 0
for i,ampli_ in enumerate(ampli_seq[8:]):
    #print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_].flatten()
    
    ddiff_ = dblue_-dred_
    mean_diff = np.average(ddiff_)
    mean_0 = np.average(d0_)

    if i%ncols==0 and i!=0:
        j += 1
    i = i-ncols*j
    #print(j,i)
    
    axs[j,i].hist(ddiff_,bins=300,range=(-0.04,0.04))
    axs[j,i].axvline(x=0.,ls='-',color='r')
    axs[j,i].axvline(x=mean_diff,ls='--',color='k',label=r'$\mu_{diff} = $'+'{0:.4f}'.format(mean_diff))
    axs[j,i].axvline(x=mean_diff+mean_0*percent/100.,ls=':',color='k',label=r'$\mu_{diff} \pm $'+'{0}'.format(percent/100.)+r'$\mu_0$')
    axs[j,i].axvline(x=mean_diff-mean_0*percent/100.,ls=':',color='k')
    axs[j,i].grid()
    axs[j,i].set_xlabel('{0}-{1}'.format(filter_blue_short,filter_red_short))
    axs[j,i].set_ylabel('Number of pixels')
    axs[j,i].set_title(ampli_)
    axs[j,i].legend(loc="best")
    
    if j!=nrows-1:
        axs[j,i].set_xlabel('')
    if i!=0:
        axs[j,i].set_ylabel('')
        
plt.savefig(os.path.join(outdir,'diff_histograms_bottom_paper.png'))


### 2D plots 

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)

color_dict_ = {0:'k',1:'white'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = ['Out','In']

percent = 1.0

diff_array = np.zeros(flat_shape).astype('int')
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_]
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_]

    ddiff_ = dblue_-dred_
    mean_diff_ = np.average(ddiff_.flatten())
    mean_0_ = np.average(d0_)

    #############################
    xmin_diff_ = np.max(ddiff_[ddiff_<=mean_diff_-mean_0_*percent/100.])
    xmax_diff_ = np.min(ddiff_[ddiff_>=mean_diff_+mean_0_*percent/100.])
    
    diff_mask_ = (ddiff_>=xmin_diff_)*(ddiff_<=xmax_diff_)

    diff_array[y0_:y1_,x0_:x1_] = diff_mask_
    
im = ax.imshow(diff_array,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
ax.set_xticklabels(ax.get_xticklabels(),fontsize=14)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=14)
cbar = fig.colorbar(im,ax=ax)
yticks = np.linspace(*cbar.ax.get_ylim(), cm_.N+1)[:-1]
yticks += (yticks[1] - yticks[0])/2
cbar.set_ticks(yticks,labels=mask_labels_)
cbar.ax.tick_params(labelsize=16)
ax.set_title(r'$\mu_{diff} \pm$'+'{0:.2f}'.format(percent/100.)+r'$\mu_0$',fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(outdir,'flat_diff_{0:.2f}percent.png'.format(percent)))


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)

color_dict_ = {-1:'r',0:'k',1:'b'}
cm_ = ListedColormap(colors=list(color_dict_.values()))
mask_labels_ = [r'$[\mu_{diff}-$'+'{0}'.format(percent/100.)+r'$\mu_0, \mu_{diff} ]$',
                'Out',
                r'$[\mu_{diff}, \mu_{diff}+$'+'{0}'.format(percent/100.)+r'$\mu_0]$']

percent = 1.0

regions_array = np.zeros(flat_shape).astype('int')
for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_]
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_]
    d0_ = flat_0[1].data[y0_:y1_,x0_:x1_]

    ddiff_ = dblue_-dred_
    mean_diff_ = np.average(ddiff_.flatten())
    mean_0_ = np.average(d0_)

    #############################
    xmin_diff_ = np.max(ddiff_[ddiff_<=mean_diff_-mean_0_*percent/100.])
    xmax_diff_ = np.min(ddiff_[ddiff_>=mean_diff_+mean_0_*percent/100.])
    
    sides_mask_ = np.zeros(ampli_shape).astype('int')
    left_mask_ = (ddiff_>=xmin_diff_)*(ddiff_<=mean_diff_)
    right_mask_ = (ddiff_>mean_diff_)*(ddiff_<=xmax_diff_)
    
    regions_array[y0_:y1_,x0_:x1_][left_mask_] = -1
    regions_array[y0_:y1_,x0_:x1_][right_mask_] = 1
    
im = ax.imshow(regions_array,cmap=cm_,origin='lower')#,vmin=0,vmax=1)
ax.set_xticklabels(ax.get_xticklabels(),fontsize=14)
ax.set_yticklabels(ax.get_yticklabels(),fontsize=14)

cbar = fig.colorbar(im,ax=ax)
yticks = np.linspace(*cbar.ax.get_ylim(), cm_.N+1)[:-1]
yticks = [yticks[1]*1.45,(yticks[1]+(yticks[1]-yticks[0])/2)*0.95,(yticks[2]+(yticks[1]-yticks[0]))*0.9]
cbar.set_ticks(yticks)
cbar.set_ticklabels(ticklabels=mask_labels_,rotation=90)
cbar.ax.tick_params(labelsize=16)
ax.set_title(r'$\mu_{diff} \pm$'+'{0:.2f}'.format(percent/100.)+r'$\mu_0$',fontsize=18)
plt.tight_layout()
plt.savefig(os.path.join(outdir,'flat_regions_{0:.2f}percent.png'.format(percent)))


## Plot flat ratios 

In [ ]:
%matplotlib inline

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]

    fig = plt.figure(figsize=(12,7))
    axs = fig.add_subplot(131)
    axs.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=0.98,vmax=1.02)
    axs.set_title(ampli_+', '+filter_blue)
    axs = fig.add_subplot(132)
    axs.imshow(flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=0.98,vmax=1.02)
    axs.set_title(ampli_+', '+filter_red)
    axs = fig.add_subplot(133)
    im = axs.imshow(flat_blue[1].data[y0_:y1_,x0_:x1_]/flat_red[1].data[y0_:y1_,x0_:x1_],cmap="gray",origin='lower',vmin=0.98,vmax=1.02)
    axs.set_title(ampli_+', ratio')
    fig.colorbar(im,ax=axs)
    

# PCA of flats 

In [ ]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
import pandas as pd

In [ ]:
def screeplot(pca, standardised_values):
    y = np.std(pca.transform(standardised_values), axis=0)**2
    x = np.arange(len(y)) + 1
    fig = plt.figure(figsize=(20,10))
    fig.set_tight_layout(True)
    plt.plot(x, y, "o-")
    plt.xticks(x, ["PC"+str(i) for i in x], ha='right', rotation=50, fontsize=12)
    plt.ylabel("Variance")
    plt.grid()
    plt.show()
    plt.close()


In [ ]:
def screeplot2(pca, standardised_values, ax):
    y = np.std(pca.transform(standardised_values), axis=0)**2
    x = np.arange(len(y)) + 1
    ax.plot(x, y, "o-")
    ax.grid()
    ax.set_xticks(x, ["PC"+str(i) for i in x], ha='right', rotation=50, fontsize=12)
    ax.set_ylabel("Variance")
    #plt.show()
    #plt.close()


In [ ]:
def pca_summary(pca, standardised_data, out=True):
    names = ["PC"+str(i) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    a = list(np.std(pca.transform(standardised_data), axis=0))
    b = list(pca.explained_variance_ratio_)
    c = [np.sum(pca.explained_variance_ratio_[:i]) for i in range(1,len(pca.explained_variance_ratio_)+1)]
    columns = pd.MultiIndex.from_tuples([("sdev", "Standard deviation"), ("varprop", "Proportion of Variance"), ("cumprop", "Cumulative Proportion")])
    summary = pd.DataFrame(zip(a, b, c), index=names, columns=columns)
    if out:
            print("Importance of components:")
            display(summary)
    return summary


In [ ]:
flat_dict_ = {}

In [ ]:
x0_ = ampli_coords[ampli_][0]
x1_ = ampli_coords[ampli_][1]
y0_ = ampli_coords[ampli_][2]
y1_ = ampli_coords[ampli_][3]

dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_]
dred_ = flat_red[1].data[y0_:y1_,x0_:x1_]

flat_dict_['blue'] = dblue_.ravel()
flat_dict_['red'] = dred_.ravel()

In [ ]:
flat_df = pd.DataFrame(flat_dict_)

In [ ]:
flat_df.head(5)

In [ ]:
standard = scale(flat_df)

In [ ]:
standard.shape

In [ ]:
standard_df = pd.DataFrame(standard,columns=flat_df.columns)
standard_df.head(5)

In [ ]:
pca = PCA().fit(standard_df)

In [ ]:
screeplot(pca,standard_df)

In [ ]:
summary_ = pca_summary(pca,standard_df)

In [ ]:

pca_dict = {}
for i,ampli_ in enumerate(ampli_seq):
    print(ampli_)
    
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]
    
    dblue_ = flat_blue[1].data[y0_:y1_,x0_:x1_].flatten()
    dred_ = flat_red[1].data[y0_:y1_,x0_:x1_].flatten()
    
    ampli_dict_ = {'blue':dblue_,'red':dred_}
    ampli_df_ = pd.DataFrame(ampli_dict_)
    
    standard_ = scale(ampli_df_)
    standard_df_ = pd.DataFrame(standard_,columns=ampli_df_.columns)
    pca_ = PCA().fit(standard_df_)
    pca_dict[ampli_] = (pca_,standard_df_)
    

In [ ]:
pca_dict['C00'][0]

In [ ]:
pca_dict['C00'][1].head(5)

In [ ]:
fig = plt.figure(figsize=(12,7))
ax = fig.add_subplot(111)

for i,ampli_ in enumerate(ampli_seq):
    var_pca1_,var_pca2_ = pca_dict[ampli_][0].explained_variance_ratio_
    ax.scatter(i,var_pca1_,marker='o',color='orange')
    ax.scatter(i,var_pca2_,marker='o',color='purple')
ax.grid()
ax.set_xlabel('Amplifier')
ax.set_ylabel('Explained variance ratio')
ax.set_xticks(np.arange(len(ampli_seq)),labels=ampli_seq)

In [ ]:
flat_pca1 = np.zeros(flat_blue[1].data.shape)
flat_pca2 = np.zeros(flat_blue[1].data.shape)

for i,ampli_ in enumerate(ampli_seq):
    x0_ = ampli_coords[ampli_][0]
    x1_ = ampli_coords[ampli_][1]
    y0_ = ampli_coords[ampli_][2]
    y1_ = ampli_coords[ampli_][3]

    pca_ = pca_dict[ampli_][0]
    standard_df_ = pca_dict[ampli_][1]
    pca1_values_ = pca_.transform(standard_df_)[:,0]
    pca2_values_ = pca_.transform(standard_df_)[:,1]

    pca1_values_ = pca1_values_/np.median(pca1_values_)
    pca2_values_ = pca2_values_/np.median(pca2_values_)
    
    pca1_values_ = pca1_values_.reshape(ampli_shape)
    pca2_values_ = pca2_values_.reshape(ampli_shape)

    flat_pca1[y0_:y1_,x0_:x1_] = pca1_values_
    flat_pca2[y0_:y1_,x0_:x1_] = pca2_values_
    
    

In [ ]:
fig,axs = plt.subplots(nrows=2,ncols=2,figsize=(12,12))

im1 = axs[0,0].imshow(flat_blue[1].data,cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
axs[0,0].set_title(filter_blue_short)
fig.colorbar(im1,ax=axs[0,0])

im2 = axs[0,1].imshow(flat_red[1].data,cmap="gray",origin='lower',vmin=fval_min,vmax=fval_max)
axs[0,1].set_title(filter_red_short)
fig.colorbar(im2,ax=axs[0,1])

im3 = axs[1,0].imshow(flat_pca1,cmap="gray",origin='lower')#,vmin=fval_min,vmax=fval_max)
axs[1,0].set_title('PC-1')
fig.colorbar(im3,ax=axs[1,0])

im4 = axs[1,1].imshow(flat_pca2,cmap="gray",origin='lower')#,vmin=fval_min,vmax=fval_max)
axs[1,1].set_title('PC-2')
fig.colorbar(im4,ax=axs[1,1])

In [ ]:
fig = plt.figure(figsize=(14,7))
ax1 = fig.add_subplot(111)
im = ax1.imshow(flat_pca1,cmap="gray",origin='lower',vmin=-100,vmax=100)
ax1.set_title('PC-1')
fig.colorbar(im,ax=ax1)

In [ ]:
fig = plt.figure(figsize=(14,7))
ax1 = fig.add_subplot(111)
im = ax1.imshow(flat_pca2,cmap="gray",origin='lower',vmin=-50,vmax=50)
ax1.set_title('PC-2')
fig.colorbar(im,ax=ax1)